# Regression in TensorFlow

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()
warnings.simplefilter(action='ignore', category=ImportWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=ResourceWarning)

In [ ]:
import tensorflow as tf

## Steps in fittting a model

- Define model variables (parameters) and placeholders (data)
- Define the loss function
- Choose an optimizer to minimize the loss
- Start a session
    - Initialize global variables
    - Run the optimizer for $n$ steps or epochs, feeding in appropriate data

## Linear Regression

In [ ]:
np.random.seed(123)

In [ ]:
N = 10
W_true = 2
b_true = 1
X_obs = np.arange(N).reshape((-1,1))
eps = np.random.normal(0, 1, (N, 1))
y_obs = np.reshape(W_true * X_obs + b_true + eps, (-1, 1))

In [ ]:
plt.scatter(X_obs, y_obs)
plt.plot(X_obs, W_true * X_obs + b_true, c='red')
pass

In [ ]:
X = tf.placeholder(tf.float32, (N, 1))
y = tf.placeholder(tf.float32, (N, 1))
W = tf.Variable(tf.random_normal((1,1)))
b = tf.Variable(tf.random_normal((1,)))
yhat = tf.matmul(X, W) + b
loss = tf.reduce_sum(tf.square(y - yhat))

train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)
init_op = tf.global_variables_initializer()

In [ ]:
niter = 1001

with tf.Session() as sess:
    sess.run(init_op)
    for i in range(niter):
        _, weights, bias, l = sess.run([train_op, W, b, loss], feed_dict={X: X_obs, y: y_obs})
        if i % 100 == 0:
            print('%03d\t%6.2f\t%6.2f\t%6.2f' % (i, weights[0][0], bias[0], l))

## Logistic Regression

We will use logistic regreesion to predict entry to graduate school based on GRE, GPA and rank of undegraduate college by prestige (1 = highest, 4= lowest).

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://stats.idre.ucla.edu/stat/data/binary.csv')

In [ ]:
df.head()

In [ ]:
df = pd.get_dummies(df, columns=['rank'], drop_first=True)
df.head()

Reset the data flow graph.

In [ ]:
N = df.shape[0]
X = tf.placeholder(tf.float32, (N, 5))
y = tf.placeholder(tf.float32, (N, 1))

W = tf.Variable(tf.random_normal((5,1)))
b = tf.Variable(tf.random_normal((1,)))

yhat = tf.matmul(X, W) + b

In [ ]:
loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=yhat, labels=y))

In [ ]:
train_op = tf.train.AdamOptimizer().minimize(loss)
init_op = tf.global_variables_initializer()

In [ ]:
niter = 25001

with tf.Session() as sess:
    sess.run(init_op)
    for i in range(niter):
        _, weights, bias, l = sess.run([train_op, W, b, loss], feed_dict={X: df.iloc[:, 1:], y: df.iloc[:, 0:1]})
        if i % 5000 == 0:
            print((i, weights.T[0], bias[0], l))

#### R fit for comparison

```R
## Call:
## glm(formula = admit ~ gre + gpa + rank, family = "binomial", 
##     data = mydata)
## 
## Deviance Residuals: 
##    Min      1Q  Median      3Q     Max  
## -1.627  -0.866  -0.639   1.149   2.079  
## 
## Coefficients:
##             Estimate Std. Error z value Pr(>|z|)    
## (Intercept) -3.98998    1.13995   -3.50  0.00047 ***
## gre          0.00226    0.00109    2.07  0.03847 *  
## gpa          0.80404    0.33182    2.42  0.01539 *  
## rank2       -0.67544    0.31649   -2.13  0.03283 *  
## rank3       -1.34020    0.34531   -3.88  0.00010 ***
## rank4       -1.55146    0.41783   -3.71  0.00020 ***
## ---
## Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
## 
## (Dispersion parameter for binomial family taken to be 1)
## 
##     Null deviance: 499.98  on 399  degrees of freedom
## Residual deviance: 458.52  on 394  degrees of freedom
## AIC: 470.5
## 
## Number of Fisher Scoring iterations: 4
```